In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, "Informer-Tensorflow/models")
sys.path.insert(1, "Informer-Tensorflow")
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime

In [2]:
from attn import ProbAttention, AttentionLayer

In [3]:
class MultiHeadProbAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadProbAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.attn = AttentionLayer(ProbAttention(False), self.d_model, self.num_heads)
    def call(self, inputs):
        query = inputs
        key = inputs
        value = inputs
        x = self.attn([query, key, value])
        return x


In [4]:
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1) 

    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        # self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)

    # def compute_mask(self, *args, **kwargs):
    #     return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        # x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

In [5]:
class ConvLayer(keras.layers.Layer):
    def __init__(self, c_in):
        super(ConvLayer, self).__init__()
        self.downConv = tf.keras.layers.Conv1D(
                                  filters=c_in,
                                  kernel_size=3,
                                  padding='causal')
        self.norm = tf.keras.layers.BatchNormalization()
        self.activation = tf.keras.layers.ELU()
        self.maxPool = tf.keras.layers.MaxPool1D(pool_size=3)#, strides=2)

    def call(self, x, **kargs):
        x = self.downConv(x)
        x = self.norm(x)
        x = self.activation(x)
        x = self.maxPool(x)
        return x

In [6]:
def models():
    inputs = keras.layers.Input(shape=(2000,1), batch_size=4)
    x = keras.layers.Dense(32, activation="tanh")(inputs)
    x = PositionalEmbedding(32)(x)
    x = MultiHeadProbAttention(128, 32)(x)
    x = ConvLayer(64)(x)
    x = MultiHeadProbAttention(64, 32)(x)
    x = ConvLayer(32)(x)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.GlobalAveragePooling1D()(x)
    x = keras.layers.Dense(5)(x)
    return keras.Model(inputs, x)

In [7]:
model = models()

[TensorShape([4, 2000, 32]), TensorShape([4, 2000, 32]), TensorShape([4, 2000, 32])]
[TensorShape([4, 666, 64]), TensorShape([4, 666, 64]), TensorShape([4, 666, 64])]


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(5, 2000, 1)]            0         
_________________________________________________________________
dense (Dense)                (5, 2000, 32)             64        
_________________________________________________________________
positional_embedding (Positi (5, 2000, 32)             0         
_________________________________________________________________
multi_head_prob_attention (M (5, 2000, 128)            29184     
_________________________________________________________________
conv_layer (ConvLayer)       (5, 666, 64)              24896     
_________________________________________________________________
multi_head_prob_attention_1  (5, 666, 64)              16640     
_________________________________________________________________
conv_layer_1 (ConvLayer)     (5, 222, 32)              6304  

In [8]:
x = tf.ones([4,2000,1])
model(x)

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[-2.9617932 ,  1.779978  , -1.3469354 , -0.27951664, -1.2706252 ],
       [-2.9617932 ,  1.779978  , -1.3469354 , -0.27951664, -1.2706252 ],
       [-2.9617932 ,  1.779978  , -1.3469354 , -0.27951664, -1.2706252 ],
       [-2.9617932 ,  1.779978  , -1.3469354 , -0.27951664, -1.2706252 ]],
      dtype=float32)>

In [9]:
data = np.load("data_shuffled.npz")
X = data["X"]
Y = data["Y"]



In [10]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [11]:
learning_rate = CustomSchedule(32)
optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [12]:
model.fit(x=X[0:900], y=Y[0:900], batch_size=4, epochs=1000, validation_data=(X[900:], Y[900:]),
         callbacks=[tensorboard_callback])

Epoch 1/1000
225/225 [==============================] - 17s 66ms/step - loss: 1.5559 - accuracy: 0.2943 - val_loss: 1.2113 - val_accuracy: 0.3800
Epoch 2/1000
225/225 [==============================] - 13s 58ms/step - loss: 1.1565 - accuracy: 0.4320 - val_loss: 1.1467 - val_accuracy: 0.4000
Epoch 3/1000
225/225 [==============================] - 13s 59ms/step - loss: 1.2010 - accuracy: 0.3938 - val_loss: 1.4340 - val_accuracy: 0.4000
Epoch 4/1000
225/225 [==============================] - 13s 59ms/step - loss: 1.1555 - accuracy: 0.4006 - val_loss: 1.3056 - val_accuracy: 0.3700
Epoch 5/1000
225/225 [==============================] - 13s 59ms/step - loss: 1.1593 - accuracy: 0.4187 - val_loss: 1.1210 - val_accuracy: 0.4300
Epoch 6/1000
225/225 [==============================] - 13s 59ms/step - loss: 1.1697 - accuracy: 0.4212 - val_loss: 1.4114 - val_accuracy: 0.2800
Epoch 7/1000
225/225 [==============================] - 13s 59ms/step - loss: 1.1167 - accuracy: 0.4364 - val_loss: 1.2377 -

Epoch 56/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.8979 - accuracy: 0.5802 - val_loss: 10.7133 - val_accuracy: 0.2000
Epoch 57/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.8947 - accuracy: 0.6126 - val_loss: 4.9085 - val_accuracy: 0.4000
Epoch 58/1000
225/225 [==============================] - 14s 62ms/step - loss: 0.9046 - accuracy: 0.5635 - val_loss: 3.5511 - val_accuracy: 0.4000
Epoch 59/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.9288 - accuracy: 0.5861 - val_loss: 3.6809 - val_accuracy: 0.2700
Epoch 60/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.9203 - accuracy: 0.5848 - val_loss: 7.3998 - val_accuracy: 0.3700
Epoch 61/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.9345 - accuracy: 0.5724 - val_loss: 5.2601 - val_accuracy: 0.2000
Epoch 62/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.9638 - accuracy: 0.5677 - val_loss: 

225/225 [==============================] - 13s 60ms/step - loss: 0.8439 - accuracy: 0.6211 - val_loss: 5.2041 - val_accuracy: 0.3400
Epoch 112/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.7932 - accuracy: 0.6245 - val_loss: 6.5148 - val_accuracy: 0.2000
Epoch 113/1000
225/225 [==============================] - 13s 59ms/step - loss: 0.8143 - accuracy: 0.6074 - val_loss: 1.0484 - val_accuracy: 0.5000
Epoch 114/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.8183 - accuracy: 0.6270 - val_loss: 2.5723 - val_accuracy: 0.4100
Epoch 115/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.8387 - accuracy: 0.6149 - val_loss: 1.0384 - val_accuracy: 0.5500
Epoch 116/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.8568 - accuracy: 0.6155 - val_loss: 8.2107 - val_accuracy: 0.4000
Epoch 117/1000
225/225 [==============================] - 13s 59ms/step - loss: 0.8385 - accuracy: 0.6078 - val_loss: 6.5984 - 

225/225 [==============================] - 13s 59ms/step - loss: 0.6737 - accuracy: 0.7443 - val_loss: 3.2502 - val_accuracy: 0.3700
Epoch 167/1000
225/225 [==============================] - 13s 59ms/step - loss: 0.7172 - accuracy: 0.7108 - val_loss: 3.9783 - val_accuracy: 0.2200
Epoch 168/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.7476 - accuracy: 0.7042 - val_loss: 4.8936 - val_accuracy: 0.3700
Epoch 169/1000
225/225 [==============================] - 13s 59ms/step - loss: 0.6669 - accuracy: 0.7214 - val_loss: 1.3797 - val_accuracy: 0.4800
Epoch 170/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.6865 - accuracy: 0.7208 - val_loss: 1.2133 - val_accuracy: 0.5900
Epoch 171/1000
225/225 [==============================] - 13s 59ms/step - loss: 0.7114 - accuracy: 0.7283 - val_loss: 2.0314 - val_accuracy: 0.3800
Epoch 172/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.6624 - accuracy: 0.7618 - val_loss: 6.3103 - 

225/225 [==============================] - 13s 60ms/step - loss: 0.5627 - accuracy: 0.7992 - val_loss: 1.8978 - val_accuracy: 0.4400
Epoch 222/1000
225/225 [==============================] - 13s 59ms/step - loss: 0.5782 - accuracy: 0.7907 - val_loss: 0.9147 - val_accuracy: 0.6700
Epoch 223/1000
225/225 [==============================] - 13s 59ms/step - loss: 0.5739 - accuracy: 0.8115 - val_loss: 2.7868 - val_accuracy: 0.3900
Epoch 224/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.5732 - accuracy: 0.7928 - val_loss: 0.9616 - val_accuracy: 0.7400
Epoch 225/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.5925 - accuracy: 0.7939 - val_loss: 1.0041 - val_accuracy: 0.6100
Epoch 226/1000
225/225 [==============================] - 13s 59ms/step - loss: 0.5652 - accuracy: 0.8015 - val_loss: 2.3798 - val_accuracy: 0.4700
Epoch 227/1000
225/225 [==============================] - 13s 59ms/step - loss: 0.5537 - accuracy: 0.7793 - val_loss: 4.6043 - 

...

225/225 [==============================] - 13s 60ms/step - loss: 0.3071 - accuracy: 0.8990 - val_loss: 1.8697 - val_accuracy: 0.6000
Epoch 772/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2647 - accuracy: 0.9050 - val_loss: 0.4671 - val_accuracy: 0.8700
Epoch 773/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2967 - accuracy: 0.9042 - val_loss: 6.9187 - val_accuracy: 0.3300
Epoch 774/1000
225/225 [==============================] - 14s 60ms/step - loss: 0.3132 - accuracy: 0.8782 - val_loss: 0.9963 - val_accuracy: 0.7700
Epoch 775/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2997 - accuracy: 0.9134 - val_loss: 1.0053 - val_accuracy: 0.7100
Epoch 776/1000
225/225 [==============================] - 14s 60ms/step - loss: 0.2636 - accuracy: 0.9069 - val_loss: 1.7413 - val_accuracy: 0.6800
Epoch 777/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2849 - accuracy: 0.9208 - val_loss: 0.5385 - 

225/225 [==============================] - 14s 60ms/step - loss: 0.3000 - accuracy: 0.9053 - val_loss: 2.5701 - val_accuracy: 0.6400
Epoch 827/1000
225/225 [==============================] - 14s 60ms/step - loss: 0.3070 - accuracy: 0.8964 - val_loss: 1.9875 - val_accuracy: 0.5900
Epoch 828/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.3053 - accuracy: 0.8806 - val_loss: 0.5033 - val_accuracy: 0.8500
Epoch 829/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2489 - accuracy: 0.9027 - val_loss: 0.9507 - val_accuracy: 0.7100
Epoch 830/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2851 - accuracy: 0.8826 - val_loss: 0.4592 - val_accuracy: 0.8200
Epoch 831/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2504 - accuracy: 0.9168 - val_loss: 13.0227 - val_accuracy: 0.2400
Epoch 832/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.3179 - accuracy: 0.8877 - val_loss: 1.7293 -

225/225 [==============================] - 13s 60ms/step - loss: 0.2462 - accuracy: 0.9249 - val_loss: 2.2416 - val_accuracy: 0.6300
Epoch 882/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2892 - accuracy: 0.8990 - val_loss: 1.4343 - val_accuracy: 0.6800
Epoch 883/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2657 - accuracy: 0.9129 - val_loss: 0.5096 - val_accuracy: 0.8100
Epoch 884/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.3248 - accuracy: 0.9026 - val_loss: 2.1195 - val_accuracy: 0.6400
Epoch 885/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2938 - accuracy: 0.8978 - val_loss: 2.8435 - val_accuracy: 0.6200
Epoch 886/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.3150 - accuracy: 0.9093 - val_loss: 0.7144 - val_accuracy: 0.7900
Epoch 887/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2782 - accuracy: 0.8957 - val_loss: 0.7852 - 

225/225 [==============================] - 13s 60ms/step - loss: 0.2619 - accuracy: 0.9068 - val_loss: 0.8747 - val_accuracy: 0.8000
Epoch 937/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.5133 - accuracy: 0.8796 - val_loss: 4.2362 - val_accuracy: 0.4400
Epoch 938/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2898 - accuracy: 0.9052 - val_loss: 3.2412 - val_accuracy: 0.5100
Epoch 939/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2343 - accuracy: 0.9182 - val_loss: 4.1260 - val_accuracy: 0.4200
Epoch 940/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2433 - accuracy: 0.9151 - val_loss: 0.5971 - val_accuracy: 0.8200
Epoch 941/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2829 - accuracy: 0.9159 - val_loss: 5.9522 - val_accuracy: 0.3500
Epoch 942/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2215 - accuracy: 0.9324 - val_loss: 5.4211 - 

225/225 [==============================] - 13s 60ms/step - loss: 0.2358 - accuracy: 0.9220 - val_loss: 1.7298 - val_accuracy: 0.6900
Epoch 992/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2679 - accuracy: 0.9184 - val_loss: 6.3155 - val_accuracy: 0.2900
Epoch 993/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2488 - accuracy: 0.9022 - val_loss: 0.5841 - val_accuracy: 0.8400
Epoch 994/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2907 - accuracy: 0.9121 - val_loss: 0.8239 - val_accuracy: 0.7400
Epoch 995/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2589 - accuracy: 0.9034 - val_loss: 1.9496 - val_accuracy: 0.6100
Epoch 996/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2302 - accuracy: 0.9157 - val_loss: 7.8656 - val_accuracy: 0.3200
Epoch 997/1000
225/225 [==============================] - 13s 60ms/step - loss: 0.2296 - accuracy: 0.9149 - val_loss: 0.4519 - 